In [10]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import zillow_functions as zl
import requests

# Create list of search terms.
# Function zipcodes_list() creates a list of US zip codes that will be 
# passed to the scraper. For example, st = zipcodes_list(["10", "11", "606"])  
# will yield every US zip code that begins with "10", begins with "11", or 
# begins with "606", as a list object.
# I recommend using zip codes, as they seem to be the best option for catching
# as many house listings as possible. If you want to use search terms other 
# than zip codes, simply skip running zipcodes_list() function below, and add 
# a line of code to manually assign values to object st, for example:
# st = ["Chicago", "New Haven, CT", "77005", "Jacksonville, FL"]
# Keep in mind that, for each search term, the number of listings scraped is 
# capped at 520, so in using a search term like "Chicago" the scraper would 
# end up missing most of the results.
# Param st_items can be either a list of zipcode strings, or a single zipcode 
# string.
st = zl.zipcodes_list(st_items = ["98296", "98290", "98208", "98205", "98012", "98072", "98011", "98021",
                                  "98205", "98258", "98271", "98223", "98292", "98274", "98273", "98233",
                                  "98232", "98229", "98225", "98226", "98248", "98264", "98240", "98230"])

# Initialize the webdriver.
driver = zl.init_driver("/Users/gilliangoodman/Downloads/chromedriver")

# Go to www.zillow.com/homes
zl.navigate_to_website(driver, "http://www.zillow.com/homes")

# Click the "buy" button.
zl.click_buy_button(driver)

# Get total number of search terms.
num_search_terms = len(st)

# Initialize list obj that will house all scraped data.
output_data = []

# Start the scraping.
for idx, term in enumerate(st):
    # Enter search term and execute search.
    if zl.enter_search_term(driver, term):
        print("Entering search term %s of %s" % 
              (str(idx + 1), str(num_search_terms)))
    else:
        print("Search term %s failed, moving on to next search term\n***" % 
              str(idx + 1))
        continue

    # Check to see if any results were returned from the search.
    # If there were none, move onto the next search.
    if zl.test_for_no_results(driver):
        print("Search %s returned zero results. Moving on to next search\n***" %
              str(term))
        continue

    # Pull the html for each page of search results. Zillow caps results at 
    # 20 pages, each page can contain 26 home listings, thus the cap on home 
    # listings per search is 520.
    raw_data = zl.get_html(driver)
    print("%s pages of listings found" % str(len(raw_data)))

    # Take the extracted HTML and split it up by individual home listings.
    listings = zl.get_listings(raw_data)
    print("%s home listings scraped\n***" % str(len(listings)))

    # For each home listing, extract the 11 variables that will populate that 
    # specific observation within the output dataframe.
    for home in listings:
        soup = BeautifulSoup(home, "lxml")
        new_obs = []

        # List that contains number of beds, baths, and total sqft (and 
        # sometimes price as well).
        card_info = zl.get_card_info(soup)

        # Street Address
        new_obs.append(zl.get_street_address(soup))
        
        # City
        new_obs.append(zl.get_city(soup))
        
        # State
        new_obs.append(zl.get_state(soup))
        
        # Zipcode
        new_obs.append(zl.get_zipcode(soup))
        
        # Price
        new_obs.append(zl.get_price(soup, card_info))
        
        # Sqft
        new_obs.append(zl.get_sqft(card_info))
        
        # Bedrooms
        new_obs.append(zl.get_bedrooms(card_info))
        
        # Bathrooms
        new_obs.append(zl.get_bathrooms(card_info))
        
        # Days on the Market/Zillow
        new_obs.append(zl.get_days_on_market(soup))
        
        # Sale Type (House for Sale, New Construction, Foreclosure, etc.)
        new_obs.append(zl.get_sale_type(soup))
        
        # URL for each house listing
        new_obs.append(zl.get_url(soup))
        
        # Append new_obs to list output_data.
        output_data.append(new_obs)

# Close the webdriver connection.
zl.close_connection(driver)

Entering search term 1 of 24
2 pages of listings found
49 home listings scraped
***
Entering search term 2 of 24
5 pages of listings found
110 home listings scraped
***
Entering search term 3 of 24
3 pages of listings found
74 home listings scraped
***
Entering search term 4 of 24
Search 98205 returned zero results. Moving on to next search
***
Entering search term 5 of 24
5 pages of listings found
114 home listings scraped
***
Entering search term 6 of 24
3 pages of listings found
73 home listings scraped
***
Entering search term 7 of 24
2 pages of listings found
48 home listings scraped
***
Entering search term 8 of 24
3 pages of listings found
59 home listings scraped
***
Entering search term 9 of 24
Search 98205 returned zero results. Moving on to next search
***
Entering search term 10 of 24
6 pages of listings found
133 home listings scraped
***
Entering search term 11 of 24
5 pages of listings found
107 home listings scraped
***
Entering search term 12 of 24
6 pages of listings 

In [11]:
# Write data to data frame, then to CSV file.
file_name = "%s_%s.csv" % (str(time.strftime("%Y-%m-%d")), 
                           str(time.strftime("%H%M%S")))
columns = ["address", "city", "state", "zip", "price", "sqft", "bedrooms", 
           "bathrooms", "days_on_zillow", "sale_type", "url"]
df = pd.DataFrame(output_data, columns = columns)

In [12]:
desc = []
df
for row in df['url']:
    url = row
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    desc.append(zl.get_description(soup))
    
df['description'] = desc

In [24]:
df_nona = df[df['description'] != 'NA']



In [26]:
len(df_nona)

942

In [32]:
df_nona = df_nona.reset_index(drop=True)

In [ ]:
df.to_csv(file_name, index = False)

In [33]:
df_nona['description'][2]

"Investor opportunity! This property is being offered at Public Auction on 05-18-2018. Visit Auction.com to see additional photos, property details, and title information (when available). Auction.com is the nation's leading real estate transaction platform focused exclusively on the sale of residential foreclosure and bank-owned properties. The majority of these properties are priced below market value. Don't miss this special opportunity to buy homes at wholesale prices! In addition to this property, 33 other properties are scheduled for sale at this same Foreclosure Sale. In our online auctions and live Foreclosure Sales, Auction.com currently has 69 properties scheduled for sale in Snohomish County and 629 throughout Washington. All properties and sale details can be found with a simple search at Auction.com. Create a FREE account today to find more properties like this one, save searches of properties that meet your investment criteria and have the properties you're looking for em

In [34]:
st = zl.zipcodes_list(st_items = ["98203", "98204", "98087", "98037", "98026", "98036", "98020", "98043",
                                  "98177", "98133", "98155", "98028", "98034", "98052", "98033", "98053",
                                  "98107", "98117", "98105", "98103", "98115", "98125", "98075", "98074"])

# Initialize the webdriver.
driver = zl.init_driver("/Users/gilliangoodman/Downloads/chromedriver")

# Go to www.zillow.com/homes
zl.navigate_to_website(driver, "http://www.zillow.com/homes")

# Click the "buy" button.
zl.click_buy_button(driver)

# Get total number of search terms.
num_search_terms = len(st)

# Initialize list obj that will house all scraped data.
output_data = []

# Start the scraping.
for idx, term in enumerate(st):
    # Enter search term and execute search.
    if zl.enter_search_term(driver, term):
        print("Entering search term %s of %s" % 
              (str(idx + 1), str(num_search_terms)))
    else:
        print("Search term %s failed, moving on to next search term\n***" % 
              str(idx + 1))
        continue

    # Check to see if any results were returned from the search.
    # If there were none, move onto the next search.
    if zl.test_for_no_results(driver):
        print("Search %s returned zero results. Moving on to next search\n***" %
              str(term))
        continue

    # Pull the html for each page of search results. Zillow caps results at 
    # 20 pages, each page can contain 26 home listings, thus the cap on home 
    # listings per search is 520.
    raw_data = zl.get_html(driver)
    print("%s pages of listings found" % str(len(raw_data)))

    # Take the extracted HTML and split it up by individual home listings.
    listings = zl.get_listings(raw_data)
    print("%s home listings scraped\n***" % str(len(listings)))

    # For each home listing, extract the 11 variables that will populate that 
    # specific observation within the output dataframe.
    for home in listings:
        soup = BeautifulSoup(home, "lxml")
        new_obs = []

        # List that contains number of beds, baths, and total sqft (and 
        # sometimes price as well).
        card_info = zl.get_card_info(soup)

        # Street Address
        new_obs.append(zl.get_street_address(soup))
        
        # City
        new_obs.append(zl.get_city(soup))
        
        # State
        new_obs.append(zl.get_state(soup))
        
        # Zipcode
        new_obs.append(zl.get_zipcode(soup))
        
        # Price
        new_obs.append(zl.get_price(soup, card_info))
        
        # Sqft
        new_obs.append(zl.get_sqft(card_info))
        
        # Bedrooms
        new_obs.append(zl.get_bedrooms(card_info))
        
        # Bathrooms
        new_obs.append(zl.get_bathrooms(card_info))
        
        # Days on the Market/Zillow
        new_obs.append(zl.get_days_on_market(soup))
        
        # Sale Type (House for Sale, New Construction, Foreclosure, etc.)
        new_obs.append(zl.get_sale_type(soup))
        
        # URL for each house listing
        new_obs.append(zl.get_url(soup))
        
        # Append new_obs to list output_data.
        output_data.append(new_obs)

# Close the webdriver connection.
zl.close_connection(driver)


CAPTCHA!
Manually complete the captcha requirements.
Once that's done, if the program was in the middle of scraping (and is still running), it should resume scraping after ~30 seconds.
Entering search term 1 of 24
3 pages of listings found
74 home listings scraped
***
Entering search term 2 of 24
3 pages of listings found
62 home listings scraped
***
Entering search term 3 of 24
3 pages of listings found
69 home listings scraped
***
Entering search term 4 of 24
2 pages of listings found
35 home listings scraped
***
Entering search term 5 of 24
3 pages of listings found
55 home listings scraped
***
Entering search term 6 of 24
2 pages of listings found
39 home listings scraped
***
Entering search term 7 of 24
2 pages of listings found
40 home listings scraped
***
Entering search term 8 of 24
1 pages of listings found
18 home listings scraped
***
Entering search term 9 of 24
2 pages of listings found
28 home listings scraped
***
Entering search term 10 of 24
2 pages of listings found
44

In [35]:
# Write data to data frame, then to CSV file.
file_name = "%s_%s.csv" % (str(time.strftime("%Y-%m-%d")), 
                           str(time.strftime("%H%M%S")))
columns = ["address", "city", "state", "zip", "price", "sqft", "bedrooms", 
           "bathrooms", "days_on_zillow", "sale_type", "url"]
df_sea = pd.DataFrame(output_data, columns = columns)

In [37]:
desc_sea = []
for row in df_sea['url']:
    url = row
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    desc_sea.append(zl.get_description(soup))
    
df_sea['description'] = desc_sea

In [38]:
df_sea

,address,city,state,zip,price,sqft,bedrooms,bathrooms,days_on_zillow,sale_type,url,description
0,3615 Shore Ave,Everett,WA,98203,2999777,9347,5,7,1,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,NA
1,2433 W Mukilteo Blvd,Everett,WA,98203,749950,2505,4,4,NA,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,NA
2,5901 Highway Pl # DF7CDP,Everett,WA,98203,279995,1175,2,3,2,New Construction,http://www.zillow.com/homes/for_sale/209180933...,NA
3,7302 7th Dr W,Everett,WA,98203,365000,1450,3,2,1,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,"This fabulous 1,450 sq ft, 3 bedroom, 1.75 bat..."
4,6010 Mcdougall Ave,Everett,WA,98203,275000,1104,3,1,1,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,NA
5,5731 12th Ave W APT I301,Everett,WA,98203,239940,966,2,1.75,1,Condo For Sale,http://www.zillow.com/homes/for_sale//homedeta...,Beautiful corner light & bright condo w/ vault...
6,1312 Ballew Ave,Everett,WA,98203,299950,2272,3,2,3,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,Investor Alert! Tons of sweat equity here. 3be...
7,6132 1st Dr SE,Everett,WA,98203,389950,1492,3,2,5,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,NA
8,5609 Broadway,Everett,WA,98203,349950,1496,3,2,3,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,NA
9,6927 Lower Ridge Rd,Everett,WA,98203,389995,2160,4,2,4,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,"Come see this well-maintained 4 Bedroom, 2 Bat..."


In [39]:
df_nona_sea = df_sea[df_sea['description'] != 'NA']

In [40]:
len(df_nona_sea)

429

In [41]:
df_nona_sea = df_nona_sea.reset_index(drop=True)

In [42]:
st = zl.zipcodes_list(st_items = ["98199", "98004", "98005", "98007", "98008", "98029", "98027", "98059",
                                  "98122", "98101", "98121", "98112", "98109", "98119", "98040", "98056",
                                  "98104", "98144", "98134", "98116", "98126", "98106", "98136", "98108"])

# Initialize the webdriver.
driver = zl.init_driver("/Users/gilliangoodman/Downloads/chromedriver")

# Go to www.zillow.com/homes
zl.navigate_to_website(driver, "http://www.zillow.com/homes")

# Click the "buy" button.
zl.click_buy_button(driver)

# Get total number of search terms.
num_search_terms = len(st)

# Initialize list obj that will house all scraped data.
output_data = []

# Start the scraping.
for idx, term in enumerate(st):
    # Enter search term and execute search.
    if zl.enter_search_term(driver, term):
        print("Entering search term %s of %s" % 
              (str(idx + 1), str(num_search_terms)))
    else:
        print("Search term %s failed, moving on to next search term\n***" % 
              str(idx + 1))
        continue

    # Check to see if any results were returned from the search.
    # If there were none, move onto the next search.
    if zl.test_for_no_results(driver):
        print("Search %s returned zero results. Moving on to next search\n***" %
              str(term))
        continue

    # Pull the html for each page of search results. Zillow caps results at 
    # 20 pages, each page can contain 26 home listings, thus the cap on home 
    # listings per search is 520.
    raw_data = zl.get_html(driver)
    print("%s pages of listings found" % str(len(raw_data)))

    # Take the extracted HTML and split it up by individual home listings.
    listings = zl.get_listings(raw_data)
    print("%s home listings scraped\n***" % str(len(listings)))

    # For each home listing, extract the 11 variables that will populate that 
    # specific observation within the output dataframe.
    for home in listings:
        soup = BeautifulSoup(home, "lxml")
        new_obs = []

        # List that contains number of beds, baths, and total sqft (and 
        # sometimes price as well).
        card_info = zl.get_card_info(soup)

        # Street Address
        new_obs.append(zl.get_street_address(soup))
        
        # City
        new_obs.append(zl.get_city(soup))
        
        # State
        new_obs.append(zl.get_state(soup))
        
        # Zipcode
        new_obs.append(zl.get_zipcode(soup))
        
        # Price
        new_obs.append(zl.get_price(soup, card_info))
        
        # Sqft
        new_obs.append(zl.get_sqft(card_info))
        
        # Bedrooms
        new_obs.append(zl.get_bedrooms(card_info))
        
        # Bathrooms
        new_obs.append(zl.get_bathrooms(card_info))
        
        # Days on the Market/Zillow
        new_obs.append(zl.get_days_on_market(soup))
        
        # Sale Type (House for Sale, New Construction, Foreclosure, etc.)
        new_obs.append(zl.get_sale_type(soup))
        
        # URL for each house listing
        new_obs.append(zl.get_url(soup))
        
        # Append new_obs to list output_data.
        output_data.append(new_obs)

# Close the webdriver connection.
zl.close_connection(driver)

Entering search term 1 of 24
2 pages of listings found
40 home listings scraped
***
Entering search term 2 of 24
4 pages of listings found
76 home listings scraped
***
Entering search term 3 of 24
1 pages of listings found
20 home listings scraped
***
Entering search term 4 of 24
1 pages of listings found
16 home listings scraped
***
Entering search term 5 of 24
2 pages of listings found
32 home listings scraped
***
Entering search term 6 of 24
3 pages of listings found
51 home listings scraped
***
Entering search term 7 of 24
3 pages of listings found
72 home listings scraped
***
Entering search term 8 of 24
3 pages of listings found
63 home listings scraped
***
Entering search term 9 of 24
3 pages of listings found
52 home listings scraped
***
Entering search term 10 of 24
2 pages of listings found
39 home listings scraped
***
Entering search term 11 of 24
2 pages of listings found
29 home listings scraped
***
Entering search term 12 of 24
2 pages of listings found
50 home listings s

In [43]:
# Write data to data frame, then to CSV file.
file_name = "%s_%s.csv" % (str(time.strftime("%Y-%m-%d")), 
                           str(time.strftime("%H%M%S")))
columns = ["address", "city", "state", "zip", "price", "sqft", "bedrooms", 
           "bathrooms", "days_on_zillow", "sale_type", "url"]
df_sea2 = pd.DataFrame(output_data, columns = columns)

In [44]:
desc_sea2 = []
for row in df_sea2['url']:
    url = row
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    desc_sea2.append(zl.get_description(soup))
    
df_sea2['description'] = desc_sea2

In [45]:
df_nona_sea2 = df_sea2[df_sea2['description'] != 'NA']

In [46]:
len(df_nona_sea2)

389

In [47]:
df_nona_sea2 = df_nona_sea2.reset_index(drop=True)

In [48]:
st = zl.zipcodes_list(st_items = ["98506", "98516", "98503", "98513", "98501", "98502", "98110", "98311",
                                  "98372", "98446", "98445", "98444", "98499", "98070", "98366", "98310",
                                  "98371", "98391", "98032", "98030", "98031", "98042", "98001", "98038",
                                  "98058"])

# Initialize the webdriver.
driver = zl.init_driver("/Users/gilliangoodman/Downloads/chromedriver")

# Go to www.zillow.com/homes
zl.navigate_to_website(driver, "http://www.zillow.com/homes")

# Click the "buy" button.
zl.click_buy_button(driver)

# Get total number of search terms.
num_search_terms = len(st)

# Initialize list obj that will house all scraped data.
output_data = []

# Start the scraping.
for idx, term in enumerate(st):
    # Enter search term and execute search.
    if zl.enter_search_term(driver, term):
        print("Entering search term %s of %s" % 
              (str(idx + 1), str(num_search_terms)))
    else:
        print("Search term %s failed, moving on to next search term\n***" % 
              str(idx + 1))
        continue

    # Check to see if any results were returned from the search.
    # If there were none, move onto the next search.
    if zl.test_for_no_results(driver):
        print("Search %s returned zero results. Moving on to next search\n***" %
              str(term))
        continue

    # Pull the html for each page of search results. Zillow caps results at 
    # 20 pages, each page can contain 26 home listings, thus the cap on home 
    # listings per search is 520.
    raw_data = zl.get_html(driver)
    print("%s pages of listings found" % str(len(raw_data)))

    # Take the extracted HTML and split it up by individual home listings.
    listings = zl.get_listings(raw_data)
    print("%s home listings scraped\n***" % str(len(listings)))

    # For each home listing, extract the 11 variables that will populate that 
    # specific observation within the output dataframe.
    for home in listings:
        soup = BeautifulSoup(home, "lxml")
        new_obs = []

        # List that contains number of beds, baths, and total sqft (and 
        # sometimes price as well).
        card_info = zl.get_card_info(soup)

        # Street Address
        new_obs.append(zl.get_street_address(soup))
        
        # City
        new_obs.append(zl.get_city(soup))
        
        # State
        new_obs.append(zl.get_state(soup))
        
        # Zipcode
        new_obs.append(zl.get_zipcode(soup))
        
        # Price
        new_obs.append(zl.get_price(soup, card_info))
        
        # Sqft
        new_obs.append(zl.get_sqft(card_info))
        
        # Bedrooms
        new_obs.append(zl.get_bedrooms(card_info))
        
        # Bathrooms
        new_obs.append(zl.get_bathrooms(card_info))
        
        # Days on the Market/Zillow
        new_obs.append(zl.get_days_on_market(soup))
        
        # Sale Type (House for Sale, New Construction, Foreclosure, etc.)
        new_obs.append(zl.get_sale_type(soup))
        
        # URL for each house listing
        new_obs.append(zl.get_url(soup))
        
        # Append new_obs to list output_data.
        output_data.append(new_obs)

# Close the webdriver connection.
zl.close_connection(driver)

Entering search term 1 of 25
3 pages of listings found
52 home listings scraped
***
Entering search term 2 of 25
6 pages of listings found
144 home listings scraped
***
Entering search term 3 of 25
4 pages of listings found
88 home listings scraped
***
Entering search term 4 of 25
7 pages of listings found
161 home listings scraped
***
Entering search term 5 of 25
4 pages of listings found
100 home listings scraped
***
Entering search term 6 of 25
5 pages of listings found
117 home listings scraped
***
Entering search term 7 of 25
3 pages of listings found
70 home listings scraped
***
Entering search term 8 of 25
3 pages of listings found
53 home listings scraped
***
Entering search term 9 of 25
4 pages of listings found
85 home listings scraped
***
Entering search term 10 of 25
2 pages of listings found
42 home listings scraped
***
Entering search term 11 of 25
5 pages of listings found
109 home listings scraped
***
Entering search term 12 of 25
4 pages of listings found
80 home listi

In [49]:
# Write data to data frame, then to CSV file.
file_name = "%s_%s.csv" % (str(time.strftime("%Y-%m-%d")), 
                           str(time.strftime("%H%M%S")))
columns = ["address", "city", "state", "zip", "price", "sqft", "bedrooms", 
           "bathrooms", "days_on_zillow", "sale_type", "url"]
df_rand = pd.DataFrame(output_data, columns = columns)

In [51]:
desc_rand = []
for row in df_rand['url']:
    url = row
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    desc_rand.append(zl.get_description(soup))
    
df_rand['description'] = desc_rand

In [52]:
df_nona_rand = df_rand[df_rand['description'] != 'NA']

In [53]:
len(df_nona_rand)

879

In [54]:
df_nona_rand = df_nona_rand.reset_index(drop=True)

In [56]:
df_nona_rand.tail()

,address,city,state,zip,price,sqft,bedrooms,bathrooms,days_on_zillow,sale_type,url,description
874,SE 192nd St & 124th Ave SE # MWHZPE,Renton,WA,98058,575990,2733,4,3,NA,New Construction,http://www.zillow.com/homes/for_sale/209240682...,With spectacular views of Mount Rainier and th...
875,SE 192nd St & 124th Ave SE # GVMZLK,Renton,WA,98058,607990,2973,4,3,NA,New Construction,http://www.zillow.com/homes/for_sale/209240681...,With spectacular views of Mount Rainier and th...
876,18132 SE Petrovitsky Rd,RENTON,WA,98058,1000000,2800,4,3,200,Make Me Move®,http://www.zillow.com/homes/for_sale//homedeta...,Idyllic estate nestled on just shy of 8 acres ...
877,174 126th Ave SE,Renton,WA,98058,395000,NA,NA,NA,224,Lot/Land For Sale,http://www.zillow.com/homes/for_sale//homedeta...,"Level almost 1 acres lot, zone 10, Near the ne..."
878,15773 142ND Pl SE,RENTON,WA,98058,528000,2700,4,3,1786,Foreclosed,http://www.zillow.com/homes/for_sale//homedeta...,Zillow is incorrectly listing the sales histor...


In [ ]:
df_nona_rand + df_nona + df_nona_sea + df_nona_sea2

In [59]:
df_nona = df_nona.append(df_nona_sea, ignore_index=True)
df_nona = df_nona.append(df_nona_sea2, ignore_index=True)
df_nona = df_nona.append(df_nona_rand, ignore_index=True)
len(df_nona)

2639

There are only 6,000 homes for sale in Washington and I wanted to limit my search to just western Washington.

In [60]:
df_nona

,address,city,state,zip,price,sqft,bedrooms,bathrooms,days_on_zillow,sale_type,url,description
0,20014 59th Ave SE,Snohomish,WA,98296,1868000,7359,4,6,NA,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,Walk-through Video: www.seetheproperty.com/250...
1,13506 70th Dr SE,Snohomish,WA,98296,674950,2957,4,3,3,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,Wonderful Highlands East home that has been up...
2,14313 214th St SE,Snohomish,WA,98296,546000,2237,3,3,6,Auction,http://www.zillow.com/homes/for_sale//homedeta...,Investor opportunity! This property is being o...
3,11621 60th Ave SE,Snohomish,WA,98296,489950,2213,4,3,4,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,"Newer 4 br, 2.5 bths 2-story backs to greenbel..."
4,13325 81st Ave SE,Snohomish,WA,98296,720000,3091,4,3,12,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,Beautiful Murray Franklin re-sale in the desir...
5,10406 180th St SE,Snohomish,WA,98296,250000,1080,3,2,54,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,Manufactured home on 1/2 acre with territorial...
6,0 164 St SE,Snohomish,WA,98296,87710,NA,NA,NA,117,Lot/Land For Sale,http://www.zillow.com/homes/for_sale//homedeta...,Buyer to verify all possible uses with the cou...
7,11809 202nd St SE,Snohomish,WA,98296,756000,3016,4,2.5,221,Foreclosed,http://www.zillow.com/homes/for_sale//homedeta...,This 3016 square foot single family home has 4...
8,11804 Seattle Hill Rd,Snohomish,WA,98296,749732,3100,4,3,NA,House For Sale,http://www.zillow.com/homes/for_sale//homedeta...,"Luxurious, modern design new construction 3100..."
9,132 Pl Se 77th Ave SE,Snohomish,WA,98296,315000,NA,NA,NA,174,Lot/Land For Sale,http://www.zillow.com/homes/for_sale//homedeta...,Great opportunity to purchase land in the Nort...
